In [2]:
import lightning.pytorch as pl
from wav2vec2 import RequestsDataset, EmotionDataset, AudioModel
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from bert import TranscriptData, Classifier
from sklearn.metrics import recall_score

tqdm.pandas()
train_df = pd.read_csv('HCC/lab/train.csv')
develdf = pd.read_csv('HCC/lab/devel.csv')
testdf = pd.read_csv('HCC/lab/test.csv')

In [3]:

le = RequestsDataset.get_le(train_df,target='request')
devel = RequestsDataset(develdf, data_path='HCC/wav/',le=le,max_sec=30,test=True,target='request')
test = RequestsDataset(testdf, data_path='HCC/wav/',max_sec=30, test=True,target='request')

Loading and processing audio


100%|██████████| 3084/3084 [00:07<00:00, 439.50it/s]


Loading and processing audio


100%|██████████| 3503/3503 [00:07<00:00, 455.39it/s]


In [7]:
ckpt_path = 'req_ckpts/wav2vec2-com.ckpt'
model = AudioModel(2)
devel_loader = torch.utils.data.DataLoader(devel, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size=16, shuffle=False)

trainer = pl.Trainer(
    devices=1,
    accelerator='gpu'
)

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-french were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 ckpt_path = 'req_ckpts/wav2vec2-com.ckpt'                                                   │
│    2 model = AudioModel(2)                                                                       │
│ ❱  3 devel_loader = torch.utils.data.DataLoader(devel, batch_size=16, shuffle=False)             │
│    4 test_loader = torch.utils.data.DataLoader(test, batch_size=16, shuffle=False)               │
│    5                                                                                             │
│    6 trainer = pl.Trainer(                                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'devel' is not defined

In [5]:
y = devel.labels

preds = trainer.predict(model,devel_loader,ckpt_path=ckpt_path)
y_hat = torch.cat(preds).detach().cpu().numpy()
print(recall_score(y,y_hat.argmax(axis=-1),average='macro'))

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at req_ckpts/wav2vec2-com.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
Loaded model weights from the checkpoint at req_ckpts/wav2vec2-com.ckpt
/home/siddhant2021565/miniconda3/envs/COMPARE/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

In [9]:
test_preds = trainer.predict(model,test_loader,ckpt_path=ckpt_path)
test_preds = torch.cat(test_preds).detach().cpu().numpy()
testdf['request'] = le.inverse_transform(test_preds.argmax(axis=-1))

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at req_ckpts/wav2vec2-com.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
Loaded model weights from the checkpoint at req_ckpts/wav2vec2-com.ckpt
/home/siddhant2021565/miniconda3/envs/COMPARE/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

In [3]:
testdf.to_csv('HCC/lab/test1.csv',index=False)
bert_model = Classifier.load_from_checkpoint('distilbert_com/distil-com.ckpt')
isinstance(bert_model,pl.LightningModule)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_l

False

In [4]:

traindata = TranscriptData(train_df, transcript_path='HCC/transcripts',target='complaint')
devdata = TranscriptData(develdf, transcript_path='HCC/transcripts',test=True,target='complaint',le=traindata.le)
testdata = TranscriptData(testdf, transcript_path='HCC/transcripts',test=True,target='complaint')

100%|██████████| 3503/3503 [00:00<00:00, 100865.31it/s]


In [7]:
devel_loader = torch.utils.data.DataLoader(devdata, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(testdata, batch_size=16, shuffle=False)

bert_model = bert_model.to('cuda:0')
preds = []
for batch in tqdm(devel_loader):
    x0, x1= batch
    x0 = {k:v.to('cuda:0') for k,v in x0.items()}
    x1 = {k:v.to('cuda:0') for k,v in x1.items()}
    y_hat = bert_model(x0,x1)
    preds.append(y_hat.detach().cpu().numpy())

100%|██████████| 193/193 [00:41<00:00,  4.61it/s]


In [9]:
labels = devdata.df['label'].values
y_hat = np.concatenate(preds).argmax(axis=-1)
print(recall_score(labels,y_hat,average='macro'))

0.5823259054627241


In [10]:
preds = []
for batch in tqdm(test_loader):
    x0, x1= batch
    x0 = {k:v.to('cuda:0') for k,v in x0.items()}
    x1 = {k:v.to('cuda:0') for k,v in x1.items()}
    y_hat = bert_model(x0,x1)
    preds.append(y_hat.detach().cpu().numpy())

100%|██████████| 219/219 [00:46<00:00,  4.74it/s]


In [12]:
y_hat = np.concatenate(preds).argmax(axis=-1)
testdf['complaint'] = traindata.le.inverse_transform(y_hat)
submission = testdf[['filename','complaint','request']]
submission.to_csv('HCC.csv',index=False)
submission.head()

,filename,complaint,request
0,test_0001.wav,no,presta
1,test_0002.wav,yes,presta
2,test_0003.wav,yes,presta
3,test_0004.wav,yes,affil
4,test_0005.wav,no,affil


In [19]:
develdf = pd.read_csv('emotions/lab/devel.csv')
testdf = pd.read_csv('emotions/lab/test.csv')
dev_emotions = EmotionDataset(develdf, data_path='emotions/raw/wav/', test=True)
test_emotions = EmotionDataset(testdf, data_path='emotions/raw/wav/', test=True)

ckpt_path = 'ckpts/epoch=6-step=9261.ckpt'
model = AudioModel(9,task='emotions')
devel_loader = torch.utils.data.DataLoader(dev_emotions, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_emotions, batch_size=16, shuffle=False)
trainer = pl.Trainer(
    devices=1,
    accelerator='gpu'
)

Loading and processing audio


100%|██████████| 9925/9925 [00:00<00:00, 12611.86it/s]


Loading and processing audio


100%|██████████| 10143/10143 [00:01<00:00, 6109.01it/s]
Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-french were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [20]:
preds = trainer.predict(model,test_loader,ckpt_path=ckpt_path)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at ckpts/epoch=6-step=9261.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
Loaded model weights from the checkpoint at ckpts/epoch=6-step=9261.ckpt
/home/siddhant2021565/miniconda3/envs/COMPARE/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

In [27]:
y_hat = torch.sigmoid(torch.cat(preds)).detach().cpu().numpy()
print(y_hat.shape)
submission = testdf[['filename']].copy()
labels = list(testdf.columns[1:])
for i in range(len(labels)):
    submission[labels[i]] = y_hat[:,i].reshape(-1)
submission.head()

(10143, 9)


,filename,Anger,Boredom,Calmness,Concentration,Determination,Excitement,Interest,Sadness,Tiredness
0,41671.wav,0.074724,0.084136,0.368613,0.036996,0.005835,0.692624,0.986331,0.034592,0.016126
1,41672.wav,0.199625,0.165747,0.392098,0.123404,0.340784,0.504053,0.376780,0.904248,0.198320
2,41673.wav,0.168761,0.575263,0.895132,0.415742,0.458657,0.088157,0.170981,0.503815,0.374095
3,41674.wav,0.647625,0.112663,0.463723,0.266283,0.754371,0.586532,0.639892,0.156683,0.098347
4,41675.wav,0.054191,0.142857,0.213714,0.094280,0.247939,0.996375,0.810094,0.117321,0.091916


In [ ]:
submission.to_csv('emotions.csv',index=False)